In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import ceil
from math import floor
import warnings
warnings.filterwarnings('ignore')
import time
import matplotlib.pyplot as plt


t=10000
m = 16
n=30


migrations= -n
orginaldf= pd.read_csv('dataset16Uniform.csv')
orginaldf=orginaldf.drop(orginaldf.columns.values[0],1)
orginaldf.head()

,period,execution,weights
0,286.0,114.0,0.398601
1,240.0,85.0,0.354167
2,201.0,88.0,0.437811
3,309.0,112.0,0.362460
4,204.0,94.0,0.460784


In [2]:
df = orginaldf.copy(deep=True)
df.head()

,period,execution,weights
0,286.0,114.0,0.398601
1,240.0,85.0,0.354167
2,201.0,88.0,0.437811
3,309.0,112.0,0.362460
4,204.0,94.0,0.460784


In [ ]:
df['remaining']=0.0
df.head()
migrations=-n
totaltime =0
Gk= 0
relDead=0
df['instance1']=0
df['instance2']=0
df['latestCPU']=-1
df['deadline']= df['period'].copy()
df['remaining'] = df['execution'].copy()
row=-1
def calculateRemaining(i):
    global df
    global totaltime
    global Gk
    global relDead
    tempdf1= df['deadline']-relDead
    x= tempdf1.min() #get ith period 
    
    Gk = x
    relDead+=Gk
    tempdf = df.copy()
    df['toexecute'] =  (tempdf['remaining']/tempdf1)*Gk
def updateTime():
    global totaltime
    global Gk
    totaltime+=Gk
def execute():
    global df
    global migrations
    executeQuantum = df['toexecute'].values
    i=0
    numCpu=0
    cpu=Gk
    
    while(i<n):
        cpu-=executeQuantum[i]
        if cpu<0:
            df['instance1'][i]=numCpu
            numCpu+=1
            cpu+=Gk
            df['instance2'][i]=numCpu
        elif cpu==0:
            df['instance2'][i]=numCpu
            df['instance1'][i]=numCpu
            numCpu+=1
            cpu+=Gk
        else:
            df['instance1'][i]=numCpu
            df['instance2'][i]=numCpu
        i+=1
    migrants=(df['instance1']^df['latestCPU']).values
    migrations+=np.count_nonzero(migrants!=0)
    migrants=(df['instance1']^df['instance2']).values
    migrations+=np.count_nonzero(migrants!=0)
    df['latestCPU']=df['instance2']
    df['remaining']-=df['toexecute']
    df['remaining']=df['remaining'].round(2)
def remakeExecuted():
    global df
    x= df[df['remaining']<=0]['remaining'].index
    tempdf= df.iloc[x,:]
    print x
    tempdf['remaining']+=tempdf['execution'].values
    tempdf['deadline']+=tempdf['period']
    df.iloc[x,:]= tempdf.copy()

In [ ]:
i =0
print df
while(totaltime<t):
#     print df
    calculateRemaining(i%n)
    execute()
    print "migrations-\t"+str(migrations)
    remakeExecuted()
    updateTime()
    i+=1
    print "Gk-\t\t"+str(Gk)
    print "time-" + str(totaltime)
    print migrations
#     print df
    

    period  execution   weights  remaining  instance1  instance2  latestCPU  \
0    286.0      114.0  0.398601      114.0          0          0         -1   
1    240.0       85.0  0.354167       85.0          0          0         -1   
2    201.0       88.0  0.437811       88.0          0          0         -1   
3    309.0      112.0  0.362460      112.0          0          0         -1   
4    204.0       94.0  0.460784       94.0          0          0         -1   
5    306.0      102.0  0.333333      102.0          0          0         -1   
6    245.0      116.0  0.473469      116.0          0          0         -1   
7    192.0      123.0  0.640625      123.0          0          0         -1   
8    193.0      105.0  0.544041      105.0          0          0         -1   
9    220.0       92.0  0.418182       92.0          0          0         -1   
10   264.0       86.0  0.325758       86.0          0          0         -1   
11   290.0       98.0  0.337931       98.0          

In [ ]:
print totaltime
print migrations